In [1]:
import numpy as np
import numpy.linalg as npla
import scipy
import scipy.linalg as sla
import pickle
import os
from sklearn.datasets import load_svmlight_file

from methods import GradientDescent, Newton, Nesterov, RBFGS, LBFGS, BFGS
from methods.rbfgs import Uniform, Gaussian, CustomDiscrete, ConstantDistribution, \
                          MatrixDistribution
from oracles import LogRegL2Oracle, create_log_reg_oracle, QuadraticOracle
from utils import select_basis_columns, run_rbfgs_experiment, run_all_methods

In [2]:
%%time
file = './datasets/SUSY'
A, b = load_svmlight_file(file, n_features=None)
print('(n, d) = {}'.format(A.shape))
print('density = {}'.format(A.count_nonzero() / (A.shape[0] * A.shape[1])))
A = A.toarray()

(n, d) = (5000000, 18)
density = 0.9882014111111111
CPU times: user 49.2 s, sys: 2 s, total: 51.1 s
Wall time: 50.9 s


In [3]:
print('Smoothness based on row norms: {}'.format(np.max(npla.norm(A, axis=1))**2 / 4.))

Smoothness based on row norms: 610.3618247607793


In [6]:
sketch_sizes = [1, 2, 5, 10, 15]
def experiment_susy(regcoef):
    oracle = create_log_reg_oracle(A, b, regcoef)
    run_all_methods(
        oracle=oracle, 
        sketch_sizes=sketch_sizes, 
        max_iter=300, 
        mat=A, 
        output_folder='./results/susy/regcoef={:.2e}'.format(regcoef), 
        sigma_tolerance=1e-8, 
        method_tolerance=1e-16, 
        stopping_criteria='grad_abs', 
        random_state=0, 
        methods=['gauss', 'coord', 'svd-no-sigma', 'bfgs', 'nesterov'], 
        overwrite=False
    )

In [7]:
%%time
experiment_susy(1e-1)

Computing SVD...Done
Singular values above tolerance: 18

RBFGS-SVD sketch no sigma... Done
RBFGS-gauss... Done
RBFGS-coord...Done
BFGS... Done

All runs completed.
CPU times: user 14h 8min 32s, sys: 35min 42s, total: 14h 44min 14s
Wall time: 27min 44s


### Nesterov

In [5]:
%%time
np.random.seed(42)
regcoef = 1e-1
oracle = create_log_reg_oracle(A, b, regcoef)
x_0 = np.random.normal(size=A.shape[1])
output_folder = './results/susy/regcoef={:.2e}'.format(regcoef)
method = Nesterov(oracle, x_0, stopping_criteria='grad_abs', 
                  tolerance=1e-16)
method.run(max_iter=500000, max_time=80)
method.oracle = None
with open('{}/nesterov.pkl'.format(output_folder), 'wb') as file:
    pickle.dump(method, file)

CPU times: user 41min 28s, sys: 1min 58s, total: 43min 26s
Wall time: 1min 21s


### Solve logreg

In [8]:
from methods import LBFGS
from tqdm import tqdm_notebook

In [9]:
def solve_logreg(A, b, regcoef, dataset_name, tolerance=1e-16):
    import os
    import pickle
    os.system('mkdir -p ./logreg_solutions/{}'.format(dataset_name))
    
    oracle = create_log_reg_oracle(A, b, regcoef)
    method = LBFGS(oracle, x_0, tolerance=tolerance, stopping_criteria='grad_abs', 
                   memory_size=10, line_search_options={'method': 'Wolfe'})
    method.run(10000)

    with open('./logreg_solutions/{}/regcoef={:.2e}'\
              .format(dataset_name, regcoef), 'wb') as file:
        pickle.dump({'x_star': method.hist['x_star'], 
                     'func_star': method.hist['func'][-1]}, file)

In [10]:
x_0 = np.zeros((A.shape[1]))

In [11]:
%%time
for regcoef in tqdm_notebook([1e-1]):
    solve_logreg(A, b, regcoef, 'susy', tolerance=1e-18)


CPU times: user 6min 36s, sys: 18.1 s, total: 6min 54s
Wall time: 13.1 s
